In [1]:
import warnings
warnings.filterwarnings('ignore')

# 교차 검증과 그리드 서치
- 머신러닝을 사용할 때, 모델의 정확도를 측정하기 위해 반드시 사용해야하는 방법.
- 딥러닝시에는 데이터의 크기가 크므로 이 방법은 사용할 필요가 없다. 


In [2]:
import pandas as pd
wine = pd.read_csv("../Data/wine.csv")
wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [3]:
# Feature, Target
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

# 검증 세트 추가

In [4]:
# 전체 세트 중 훈련세트와 테스트 세트를 8:2의 기준으로 분리한다.
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42
)

In [5]:
# 훈련세트 중 훈련세트와 검증세트를 8 : 2의 기준으로 분리한다.
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42
)

In [6]:
# 훈련세트, 검증세트, 테스트 세트의 크기 구하기
print("훈련세트 :", sub_input.shape)
print("검증세트 :", val_input.shape)
print("테스트 세트 :", test_input.shape)

훈련세트 : (4157, 3)
검증세트 : (1040, 3)
테스트 세트 : (1300, 3)


In [7]:
# 훈련세트와 검ㅈ으세트를 결정트리로 모델 만들기
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print("Train score :", dt.score(sub_input, sub_target))
print("valid score :", dt.score(val_input, val_target))

Train score : 0.9971133028626413
valid score : 0.864423076923077


---
# 교차검증

In [10]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
scores

{'fit_time': array([0.00799799, 0.00696802, 0.00674081, 0.0063448 , 0.00653291]),
 'score_time': array([0.00097895, 0.00062084, 0.00064492, 0.00118303, 0.00061798]),
 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}

In [11]:
import numpy as np
np.mean(scores['test_score'])

0.855300214703487

---
# KFold : 분할기를 사용한 교차 검증


In [19]:
from sklearn.model_selection import StratifiedKFold
splitter = StratifiedKFold(n_splits=5)     # 기본 n_splits는 5
scores = cross_validate(dt, train_input, train_target, cv=splitter)
scores

{'fit_time': array([0.00676703, 0.0060401 , 0.00659084, 0.00675011, 0.00927615]),
 'score_time': array([0.00066423, 0.00067258, 0.00088   , 0.00078988, 0.00130105]),
 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}

In [20]:
np.mean(scores['test_score'])

0.855300214703487

In [25]:
# KFold의 Fold를 10개로 나누어서 교차검증

splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)     # 기본 n_splits는 5 / shuffle로 섞는 게 더 좋다. 왜냐하면 과대적합이 되기 때문임.
scores = cross_validate(dt, train_input, train_target, cv=splitter)
scores


{'fit_time': array([0.01003098, 0.00996399, 0.008286  , 0.013484  , 0.00668406,
        0.00728178, 0.00687099, 0.00745702, 0.00704479, 0.00787616]),
 'score_time': array([0.0005672 , 0.000597  , 0.00057983, 0.00071788, 0.00057101,
        0.00062299, 0.00061607, 0.00063396, 0.00065613, 0.00069666]),
 'test_score': array([0.83461538, 0.87884615, 0.85384615, 0.85384615, 0.84615385,
        0.87307692, 0.85961538, 0.85549133, 0.85163776, 0.86705202])}

In [26]:
np.mean(scores['test_score'])


0.8574181117533719